In [ ]:
!pip install scikeras

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from keras.layers import Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
import tensorflow as tf
import random

random.seed(42)            # Python's built-in random module
np.random.seed(42)         # numpy
tf.random.set_seed(42)     # TensorFlow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
merged_d = pd.read_csv(r'/content/drive/MyDrive/Dissertation_10862121/merged_d.csv')

In [ ]:
save_dir = "r'/content/drive/MyDrive/Dissertation_10862121/"

In [ ]:
import os
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
merged_d['GMT'] = pd.to_datetime(merged_d['GMT'])
merged_d['Date'] = pd.to_datetime(merged_d['Date'])
merged_d.set_index('GMT', inplace=True)

Feature Selection

In [ ]:
merged_d = merged_d.drop(columns=["Hourly Mean Windspeed (kn)"])

In [ ]:
#merged_d['consumption_diff'] = merged_d['Total Household Consumption (D)'].diff()

Feature Engineering

Lags

Consumption

In [ ]:
#Lag for the hour before
merged_d['lag_1_hour_energy'] = merged_d['Total Household Consumption (D)'].shift(1)

#Lags for the same hour for the past 7 days
for i in range(1, 8):
    merged_d[f'lag_{i}_day_energy'] = merged_d['Total Household Consumption (D)'].shift(24 * i)

Temperature

In [ ]:
#Lag for the hour before
merged_d['lag_1_hour_temp'] = merged_d['Hourly Temperature (C)'].shift(1)

#Lags for the same hour for the past 7 days
for i in range(1, 8):
    merged_d[f'lag_{i}_day_temp'] = merged_d['Hourly Temperature (C)'].shift(24 * i)

Radiation

In [ ]:
#Lag for the hour before
merged_d['lag_1_hour_rad'] = merged_d['Hourly Global Radiation (KJ/m2)'].shift(1)

#Lags for the same hour for the past 7 days
for i in range(1, 8):
    merged_d[f'lag_{i}_day_rad'] = merged_d['Hourly Global Radiation (KJ/m2)'].shift(24 * i)

Rolling Statistics - Consumption

In [ ]:
window_size = 72
merged_d['rolling_mean_consumption'] = merged_d['Total Household Consumption (D)'].rolling(window=window_size).mean()

Cyclical Features

In [ ]:
merged_d['hour_sin'] = np.sin(2 * np.pi * merged_d['Hour'] / 24)
merged_d['hour_cos'] = np.cos(2 * np.pi * merged_d['Hour'] / 24)

merged_d['month_sin'] = np.sin(2 * np.pi * merged_d['Month'] / 12)
merged_d['month_cos'] = np.cos(2 * np.pi * merged_d['Month'] / 12)

Weather Combination Variables

In [ ]:
merged_d['temp_humidity_interaction'] = merged_d['Hourly Temperature (C)'] * merged_d['Hourly Relative Humidity (%)']

In [ ]:
merged_d['temp_rad_interaction'] = merged_d['Hourly Temperature (C)'] * merged_d['Hourly Global Radiation (KJ/m2)']

Weighted Observations (More weight to more recent)

In [ ]:
alpha = 0.9
merged_d['ewm_consumption'] = merged_d['Total Household Consumption (D)'].ewm(alpha=alpha).mean()

Dropping Missing Value Rows after Feature Engineering

In [ ]:
merged_d.dropna(inplace=True)

In [ ]:
merged_d['IsHoliday'] = merged_d['IsHoliday'].astype(int)

In [ ]:
non_num_columns = merged_d.select_dtypes(exclude=['int64', 'float64']).columns
print(non_num_columns)

Index(['Date', 'Time'], dtype='object')


In [ ]:
#Dropping columns that aren't numerical (needed for model input)
merged_d= merged_d.drop(columns=['Date', 'Time'])

In [ ]:
non_num_columns = merged_d.select_dtypes(exclude=['int64', 'float64']).columns
print(non_num_columns)

Index([], dtype='object')


Standard Scaler

In [ ]:
#Selecting numerical columns
numerical_cols = merged_d.select_dtypes(include=['float64', 'int64']).columns

#Applying StandardScaler
standard_scaler = StandardScaler()
merged_d_standard = merged_d.copy()
merged_d_standard[numerical_cols] = standard_scaler.fit_transform(merged_d[numerical_cols])

MinMaxScaler

In [ ]:
minmax_scaler = MinMaxScaler()
merged_d_minmax = merged_d.copy()
merged_d_minmax[numerical_cols] = minmax_scaler.fit_transform(merged_d[numerical_cols])

Data Preparation

Train/Test Split

In [ ]:
end_of_october = merged_d.index.get_loc('2013-10-31').stop - 1
end_of_october

7127

In [ ]:
X = merged_d_standard.drop('Total Household Consumption (D)', axis=1).values
y = merged_d_standard['Total Household Consumption (D)'].values

#Data generation function
def generate_dataset(X, y, time_steps=168, out_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps - out_steps):
        v = X[i:i + time_steps]
        Xs.append(v)
        ys.append(y[i + time_steps:i + time_steps + out_steps])
    return np.array(Xs), np.array(ys)

TIME_STEPS = 24
OUT_STEPS = 1

X_data, y_data = generate_dataset(X, y, TIME_STEPS, OUT_STEPS)

split_index = end_of_october - TIME_STEPS + 1

X_train = X_data[:split_index]
X_test = X_data[split_index:]

y_train = y_data[:split_index]
y_test = y_data[split_index:]

LSTM Build-up

In [ ]:
def create_model(optimizer='adam', lstm_units=50, dropout_rate=0.0):
    model = Sequential()
    model.add(LSTM(lstm_units, activation='relu', input_shape=(TIME_STEPS, X_data.shape[2]), return_sequences=True, dropout=dropout_rate))
    model.add(LSTM(lstm_units, activation='relu', dropout=dropout_rate))
    model.add(Dense(OUT_STEPS))
    model.compile(optimizer=optimizer, loss='mae')
    return model

In [ ]:
from sklearn.metrics import make_scorer

#MAPE function
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

#Converting the MAPE function into a scorer object
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

In [ ]:
#KerasRegressor with parameters
regressor = KerasRegressor(build_fn=create_model, optimizer='adam', lstm_units=50, dropout_rate=0.0, verbose=0)

In [ ]:
#Defining the hyperparameters for grid search
param_grid = {
    'lstm_units': [50, 70],
    'dropout_rate': [0.0, 0.2],
    'optimizer': ['adam', 'rmsprop'],
    'batch_size': [16, 32, 64],
    'epochs': [20, 30, 40]
}

In [ ]:
#Grid search setup using MAPE as the scoring function
grid = GridSearchCV(estimator=regressor, param_grid=param_grid,
                    scoring=mape_scorer, cv=2, verbose=1)

In [ ]:
grid_result = grid.fit(X_train, y_train)

Fitting 2 folds for each of 72 candidates, totalling 144 fits


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras

In [ ]:
#Grid Search results
print(f"Best score (negative MAE): {grid_result.best_score_}")
print(f"Best hyperparameters: {grid_result.best_params_}")

Best score (negative MAE): -82.94136030288362
Best hyperparameters: {'batch_size': 32, 'dropout_rate': 0.0, 'epochs': 40, 'lstm_units': 70, 'optimizer': 'adam'}


In [ ]:
import pickle

#Saving the grid search results
with open('grid_search_results.pkl', 'wb') as file:
    pickle.dump(grid_result, file)

In [ ]:
best_params = grid_result.best_params_
best_model = create_model(optimizer=best_params['optimizer'], lstm_units=best_params['lstm_units'], dropout_rate=best_params['dropout_rate'])
best_model.fit(X_train, y_train, batch_size=best_params['batch_size'], epochs=best_params['epochs'], verbose=1)
best_model.save('best_model.h5')


Epoch 1/40
222/222 [==============================] - 11s 32ms/step - loss: 0.3235
Epoch 2/40
222/222 [==============================] - 9s 40ms/step - loss: 0.1478
Epoch 3/40
222/222 [==============================] - 7s 32ms/step - loss: 0.1172
Epoch 4/40
222/222 [==============================] - 9s 40ms/step - loss: 0.1019
Epoch 5/40
222/222 [==============================] - 7s 32ms/step - loss: 0.0965
Epoch 6/40
222/222 [==============================] - 9s 40ms/step - loss: 0.0908
Epoch 7/40
222/222 [==============================] - 7s 31ms/step - loss: 0.0859
Epoch 8/40
222/222 [==============================] - 9s 40ms/step - loss: 0.0827
Epoch 9/40
222/222 [==============================] - 7s 33ms/step - loss: 0.0788
Epoch 10/40
222/222 [==============================] - 9s 38ms/step - loss: 0.0795
Epoch 11/40
222/222 [==============================] - 8s 36ms/step - loss: 0.0745
Epoch 12/40
222/222 [==============================] - 8s 36ms/step - loss: 0.0735
Epoch 13/40
